# Fixed Grid Quantization Experiment: Convergence, Computation Time, and Absolute Error Analysis

**Overview:**  
In this notebook we analyze the performance of the adapted Wasserstein distance computation using a fixed quantization grid of **0.005**.  
 
We run experiments for various sample sizes:

- **Sample Sizes:** 3000, 7000, 10000, 13000, 17000, and 20000
- **Number of Runs per Sample Size:** 10 (each with a different random seed)

**Experimental Steps:**  
1. For fixed matrices `L` and `M` (or any matrices you choose), we generate sample paths using the function `Lmatrix2paths`.  
2. For each nontrivial time step (t = 1,...,T), we apply basic grid quantization by rounding to the nearest multiple of 0.005.
3. The quantized paths are then converted into index sequences (using a unique mapping), sorted, and their conditional distributions are built.
4. The numerical adapted Wasserstein squared distance is computed using `nested2_parallel`.
 
**Outputs:**  
- **Distance Convergence:** Mean computed AW² (± standard deviation) vs. sample size with the benchmark value shown.
- **Computation Time:** Mean computation time (± standard deviation) vs. sample size.
- **Absolute Error:** Mean absolute error (|computed mean – benchmark|, ± standard deviation) vs. sample size.
 
This code is organized to be general so you can use it with other matrices or grid settings with minimal changes.

In [2]:
import os, sys, numpy as np, time, random
import matplotlib.pyplot as plt

# Set up the source path (adjust as needed for your project structure)
notebooks_path = os.path.abspath(os.getcwd())
src_path = os.path.abspath(os.path.join(notebooks_path, "../src"))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Import your custom functions
from optimal_code.utils import *
from optimal_code.optimal_solver import *
from adapted_empirical_measure.AEM_grid import *

## Experiment Setup

- **Fixed Grid Size:** 0.005  
- **Sample Sizes:** [3000, 7000, 10000, 13000, 17000, 20000]  
- **Runs per Sample Size:** 10  
- **Time Horizon (T):** 3  
- **Matrices:** (You can replace these with other matrices if needed)

We use:

$$
L = \begin{pmatrix} 1 & 0 & 0 \\ 2 & 4 & 0 \\ 3 & 2 & 1 \end{pmatrix},\quad
M = \begin{pmatrix} 1 & 0 & 0 \\ 2 & 3 & 0 \\ 3 & 1 & 2 \end{pmatrix}.
$$

In [18]:
# Experiment parameters
sample_sizes = [1000, 3000, 5000, 7500, 10000, 15000, 20000]
fixed_grid = 0.01
n_runs = 10
T = 3  # Number of time steps (excluding the trivial initial state)

# Define matrices (change these as needed for other experiments)
L = np.array([[1, 0, 0],
              [2, 4, 0],
              [3, 2, 1]])
M = np.array([[1, 0, 0],
              [2, 3, 0],
              [3, 1, 2]])

## Main Experiment Loop
 
For each sample size we perform 10 runs. In every run:
 
1. Generate sample paths for X and Y using `Lmatrix2paths` (each run uses a new random seed).
2. Compute the theoretical benchmark distance using `adapted_wasserstein_squared` (which depends only on L and M).
3. For each nontrivial time step (t = 1,...,T), apply the basic grid quantization (with grid size = 0.005).
4. Map the quantized paths to indices, sort them, build conditional distributions, and compute the numerical adapted Wasserstein squared distance.
5. Record both the computed distance and the elapsed computation time.

In [ ]:
results = {}  # Dictionary to store results keyed by sample size

for n_sample in sample_sizes:
    # Arrays to hold the results of each run (1D arrays since grid is fixed)
    distances_runs = np.zeros(n_runs)
    times_runs = np.zeros(n_runs)
    
    print(f"\nRunning experiments for sample size: {n_sample}")
    for run in range(n_runs):
        # New random seed for each run
        random_seed = np.random.randint(10000)
        
        # Generate sample paths (X for process 1 and Y for process 2)
        X, A = Lmatrix2paths(L, n_sample, seed=random_seed, verbose=False)
        Y, B = Lmatrix2paths(M, n_sample, seed=random_seed, verbose=False)
        
        # Compute the theoretical benchmark (A and B depend solely on L and M)
        benchmark = adapted_wasserstein_squared(A, B)
        
        adaptedX = path2adaptedpath(X, delta_n = fixed_grid)
        adaptedY = path2adaptedpath(Y, delta_n = fixed_grid)
        
        # Quantization map
        q2v = np.unique(np.concatenate([adaptedX, adaptedY], axis=0))
        v2q = {k: v for v, k in enumerate(q2v)}  # Value to Quantization

        # Quantized paths
        qX = np.array([[v2q[x] for x in y] for y in adaptedX])
        qY = np.array([[v2q[x] for x in y] for y in adaptedY])

        # Sort paths and transpose to (n_sample, T+1)
        qX = sort_qpath(qX.T)
        qY = sort_qpath(qY.T)

        # Get conditional distribution mu_{x_{1:t}} = mu_x[t][(x_1,...,x_t)] = {x_{t+1} : mu_{x_{1:t}}(x_{t+1}), ...}
        mu_x = qpath2mu_x(qX)
        nu_y = qpath2mu_x(qY)

        mu_x_c, mu_x_cn, mu_x_v, mu_x_w, mu_x_cumn = list_repr_mu_x(mu_x, q2v)
        nu_y_c, nu_y_cn, nu_y_v, nu_y_w, nu_y_cumn = list_repr_mu_x(nu_y, q2v)
        
        # Compute the numerical adapted Wasserstein squared distance and time the computation
        t_start = time.perf_counter()
        AW_2square = nested2_parallel(mu_x_cn, mu_x_v, mu_x_w, mu_x_cumn,
                                      nu_y_cn, nu_y_v, nu_y_w, nu_y_cumn,
                                      n_processes=30)
        elapsed = time.perf_counter() - t_start
        
        distances_runs[run] = AW_2square
        times_runs[run] = elapsed
        
        print(f"Sample size {n_sample}, Run {run+1}: AW_2^2 = {AW_2square:.4f}, Time = {elapsed:.4f} s")
    
    # Store results for this sample size
    results[n_sample] = {"distances": distances_runs,
                         "times": times_runs,
                         "benchmark": benchmark}


Running experiments for sample size: 1000


100%|██████████████████████████████████████████| 33/33 [00:00<00:00, 214.53it/s]

100%|██████████████████████████████████████████| 33/33 [00:00<00:00, 198.18it/s]

100%|███████████████████████████████████████████| 12/12 [00:00<00:00, 44.06it/s]

100%|███████████████████████████████████████████| 12/12 [00:00<00:00, 81.81it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 52.66it/s]


Sample size 1000, Run 1: AW_2^2 = 2.8918, Time = 2.2484 s


100%|██████████████████████████████████████████| 33/33 [00:00<00:00, 201.50it/s]

100%|███████████████████████████████████████████| 12/12 [00:00<00:00, 54.24it/s]

100%|██████████████████████████████████████████| 12/12 [00:00<00:00, 104.45it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 58.19it/s]


Sample size 1000, Run 2: AW_2^2 = 3.0718, Time = 2.1819 s


...

100%|█████████████████████████████████████████| 653/653 [01:08<00:00,  9.48it/s]

100%|█████████████████████████████████████████| 652/652 [01:09<00:00,  9.33it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 22.39it/s]


Sample size 20000, Run 1: AW_2^2 = 2.9918, Time = 180.4908 s


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 24.02it/s]


Sample size 20000, Run 2: AW_2^2 = 2.9414, Time = 178.2954 s


 45%|██████████████████▎                      | 291/653 [00:31<00:36,  9.81it/s]

## Aggregating Results Across Sample Sizes
 
We now compute, for each sample size:
 
 - The mean and standard deviation of the computed adapted Wasserstein squared distance.
 - The mean and standard deviation of the computation time.
 - The mean and standard deviation of the absolute error (|computed distance – benchmark|).
 
These aggregated results will be used to generate the plots.


In [ ]:
sample_list = sorted(results.keys())
mean_distances = []
std_distances = []
mean_times = []
std_times = []
mean_abs_errors = []
std_abs_errors = []
benchmarks = []  # (should be the same for all runs)

for n_sample in sample_list:
    data = results[n_sample]
    dists = data["distances"]
    times = data["times"]
    bench = data["benchmark"]
    
    mean_distances.append(np.mean(dists))
    std_distances.append(np.std(dists))
    mean_times.append(np.mean(times))
    std_times.append(np.std(times))
    abs_errors = np.abs(dists - bench)
    mean_abs_errors.append(np.mean(abs_errors))
    std_abs_errors.append(np.std(abs_errors))
    benchmarks.append(bench)

## Plotting the Results
 
We generate three plots:
 
1. **Distance Convergence:** Mean adapted Wasserstein squared distance (± std) vs. sample size with the benchmark value indicated.
2. **Computation Time:** Mean computation time (± std) vs. sample size.
3. **Absolute Error:** Mean absolute error (± std) between the computed mean distance and the benchmark vs. sample size.

In [ ]:
# Plot 1: Distance Convergence
plt.figure(figsize=(10, 6))
plt.errorbar(sample_list, mean_distances, yerr=std_distances, fmt="o-", capsize=5, label="Mean AW_2^2")
plt.axhline(y=benchmarks[0], color="red", linestyle="--", label="Benchmark")
plt.xlabel("Number of Sample Paths")
plt.ylabel("Adapted Wasserstein Squared Distance")
plt.title("Distance Convergence vs. Sample Size\n(Fixed Grid Quantization with Grid = 0.005)")
plt.legend()
plt.grid(True)
plt.show()

# Plot 2: Computation Time
plt.figure(figsize=(10, 6))
plt.errorbar(sample_list, mean_times, yerr=std_times, fmt="s-", capsize=5, color="green", label="Mean Computation Time")
plt.xlabel("Number of Sample Paths")
plt.ylabel("Computation Time (seconds)")
plt.title("Computation Time vs. Sample Size\n(Fixed Grid Quantization with Grid = 0.005)")
plt.legend()
plt.grid(True)
plt.show()

# Plot 3: Absolute Error
plt.figure(figsize=(10, 6))
plt.errorbar(sample_list, mean_abs_errors, yerr=std_abs_errors, fmt="d-", capsize=5, color="magenta", label="Mean Absolute Error")
plt.xlabel("Number of Sample Paths")
plt.ylabel("Absolute Error |Mean AW_2^2 - Benchmark|")
plt.title("Absolute Error vs. Sample Size\n(Fixed Grid Quantization with Grid = 0.005)")
plt.legend()
plt.grid(True)
plt.show()